<a href="https://www.kaggle.com/code/camillagretschel/semantic-segmentation?scriptVersionId=100352986" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np
import os 
import torch 
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import cv2
import albumentations as albu
import torch.nn as nn
import pandas as pd
!pip install -q segmentation-models-pytorch
import segmentation_models_pytorch as smp

In [ ]:
IMAGE_PATH= '../input/hubmap-256x256/train'
MASK_PATH= '../input/hubmap-256x256/masks'

In [ ]:
def create_df(path):
    name = []
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            name.append(filename.split('.')[0])
    
    return pd.DataFrame({'id': name}, index = np.arange(0, len(name)))

df = create_df(IMAGE_PATH)
print('Total Images: ', len(df))

In [ ]:
df.head()

In [ ]:
df= df.iloc[:80,]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val = train_test_split(df['id'].values, test_size=0.15)

In [ ]:
mean = [0.65459856,0.48386562,0.69428385]
std = [0.15167958,0.23584107,0.13146145]

In [ ]:
from torchvision import transforms as T
from PIL import Image

In [ ]:
class hubmap(Dataset):
    
    def __init__(self, img_path, mask_path, X, mean, std, transform=None):
        self.img_path = img_path
        self.mask_path = mask_path
        self.X = X
        self.transform = transform
        self.mean = mean
        self.std = std
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        img = cv2.imread(self.img_path +'/'+ self.X[idx] + '.png')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.mask_path +'/'+ self.X[idx] + '.png', cv2.IMREAD_GRAYSCALE)
        
        
        if self.transform is not None:
            aug = self.transform(image=img, mask=mask)
            img = Image.fromarray(aug['image'])
            mask = aug['mask']
        
        if self.transform is None:
            img = Image.fromarray(img)
        
        t = T.Compose([T.ToTensor(), T.Normalize(self.mean, self.std)])
        img = t(img)        
        mask = torch.from_numpy(mask)
        mask=torch.unsqueeze(mask,0) #Adds an extra dimension: CxHxW
        return img,mask

In [ ]:
train_transforms= albu.Compose([
            albu.HorizontalFlip(p=0.5),
            albu.VerticalFlip(p=0.5),
            albu.ShiftScaleRotate(rotate_limit=25, scale_limit=0.15, shift_limit=0, p=0.75),
            albu.CoarseDropout(max_holes=16, max_height=64 ,max_width=64 ,p=0.5),
            albu.ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.25, p=0.75),
            albu.GridDistortion(num_steps=5, distort_limit=0.3, interpolation=1, p=0.5),
    
])


In [ ]:
train_set = hubmap(IMAGE_PATH, MASK_PATH, X_train, mean, std, train_transforms)
val_set = hubmap(IMAGE_PATH, MASK_PATH, X_val, mean, std, None )

In [ ]:
train_loader = DataLoader(train_set, batch_size=10, shuffle=True)
val_loader = DataLoader(val_set, batch_size=3, shuffle=True)     

In [ ]:
import torch.nn as nn

In [ ]:
ENCODER = 'resnet34'
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'sigmoid' 

# create segmentation model with pretrained encoder
model = smp.FPN(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=1, 
    activation=ACTIVATION,
)

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

In [ ]:
loss_fn = nn.BCEWithLogitsLoss ()
optimizer= torch.optim.Adam(model.parameters(), lr=1e-3)
lr=torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, verbose=True)

In [ ]:
def training_loop(n_epochs, optimizer,lr_scheduler,model,loss_fn, train_loader, val_loader):
    tr_loss_arr = []
    val_loss_arr = []
        
    for epoch in range(0, n_epochs):
        train_loss = 0.0
        
        for X, y in train_loader:
            model.train()
            ypred = model(X)
            loss = loss_fn(ypred, y.float())
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            tr_loss_arr.append(loss.item())
            
        with torch.no_grad():
            
            val_loss = 0
            for X, y in val_loader:
                model.eval()
                ypred = model(X)
                val_loss_arr.append(loss_fn(ypred, y.float()).item())
                
        lr_scheduler.step(loss)

    return tr_loss_arr, val_loss_arr

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
history= training_loop(10,optimizer,lr,model,loss_fn,train_loader, val_loader)

In [ ]:
if torch.cuda.is_available():
    print('using device: cuda')
else:
    print('using device: cpu')

In [ ]:
train_set[0][0].shape

In [ ]:
torch.unsqueeze(train_set[0][0],0).shape